## Imports and Initialise

In [16]:
import os
import re
import csv
import json
import random
from openai import OpenAI
from dotenv import load_dotenv
from llm_generation import get_all_paths
from llm_paraphrase import initialise_prompts, get_prompt, paraphrase_mwo, process_mwo_response

load_dotenv()
api_key = os.getenv('API_KEY')
client = OpenAI(api_key=api_key)
data = get_all_paths(valid=True)

54	paths in object_property_paths
567	paths in process_agent_paths
402	paths in process_patient_paths
1743	paths in state_patient_paths
7	paths in object_property_state_paths
0	paths in object_process_state_paths
202	paths in state_agent_activity_paths
44	paths in state_agent_patient_paths
188	paths in process_agent_patient_paths
Total number of paths: 3207


## Functions

In [13]:
def get_1fewshot_message(base_prompts, instructions):
    """ Get fewshot message given fewshot csv file for single-example prompt """
    message = [{"role": "system", "content": "You are a technician recording maintenance work orders."}]
    with open("fewshot.csv", encoding='utf-8') as f:
        data = csv.reader(f)
        next(data) # Ignore header
        # Single-example prompt
        for row in data:
            if len(row) == 4:
                if row[2] == "": # No helper
                    user = {"role": "user", "content": get_prompt(base_prompts, instructions, row[0], row[1])}
                else:            # Has helper
                    user = {"role": "user", "content": get_prompt(base_prompts, instructions, row[0], row[1], row[2])}
                assistant = {"role": "assistant", "content": row[3]}
                message.append(user)
                message.append(assistant)   
    # Save fewshot message to json file
    with open("fewshot1.json", "w", encoding='utf-8') as f:
        json.dump(message, f, indent=4)
    return message

def get_5fewshot_message(base_prompts, instructions):
    """ Get fewshot message given fewshot csv file for 5-example prompt """
    message = [{"role": "system", "content": "You are a technician recording maintenance work orders."}]
    with open("fewshot.csv", encoding='utf-8') as f:
        data = csv.reader(f)
        next(data) # Ignore header
        # 5-example prompt
        for row in data:
            if len(row) > 4:
                if row[2] == "": # No helper
                    user = {"role": "user", "content": get_prompt(base_prompts, instructions, row[0], row[1])}
                else:
                    user = {"role": "user", "content": get_prompt(base_prompts, instructions, row[0], row[1], row[2])}
                example = f"1. {row[3]}\n2. {row[4]}\n3. {row[5]}\n4. {row[6]}\n5. {row[7]}"
                assistant = {"role": "assistant", "content": example}
                message.append(user)
                message.append(assistant)
    # Save fewshot message to json file
    with open("fewshot5.json", "w", encoding='utf-8') as f:
        json.dump(message, f, indent=4)
    return message


# Print some fewshot examples from MaintIE gold dataset
def print_examples(object, event, helper=None):
    """ Print some fewshot examples from the gold dataset """
    data = []
    with open("data/MaintIE/gold_release.json", encoding='utf-8') as f:
        gold = json.load(f)
        for d in gold:
            text = d['text'].replace("<id> ", "").replace(" <id>", "")
            data.append(text)
    for sentence in data:
        event_exists = re.search(rf'\b{event}\b', sentence)
        helper_exists = re.search(rf'\b{helper}\b', sentence) if helper else None
        if object in sentence and event_exists and helper_exists:
            print(f"{object},{event},{helper},{sentence}")
            return True
        elif object in sentence and event_exists:
            print(f"{object},{event},{sentence}")
            return True
    return False

# Process single response from the LLM
def process_single_response(response):
    """ Process single response from the LLM """
    response = response.lower()                     # Case folding
    response = re.sub(r'[^\w\s]', ' ', response)    # Remove punctuation
    response = re.sub(r"\s+", " ", response)        # Remove extra spaces
    return response

In [14]:
# Print some examples from the gold dataset
successful_calls = 0
while successful_calls < 5:
    current = random.choice(data)
    if 'helper_name' in current:
        if print_examples(current['object_name'], current['event_name'], current['helper_name']):
            successful_calls += 1
    else:
        if print_examples(current['object_name'], current['event_name']):
            successful_calls += 1

line,blown,blown extension line from position <num>
lube pump,leaking,differential lube pump leaking
slip ring,leak,high voltage slip ring inspection - grease leak
fan hoses,leaking,change out leaking fan hoses
fuse,keeps blowing,keeps blowing starter / horn fuse


## Same prompt VS Variant prompts

In [18]:
random.seed(42)
path = random.choice(data)
base_prompts, instructions = initialise_prompts(client, num_variants=5, num_examples=1)

APIConnectionError: Connection error.

In [42]:
# Same prompt
def same_prompt():
    outputs = []
    base_prompt = "Generate a Maintenance Work Order (MWO) sentence describing the following equipment and undesirable event."
    instruction = "The sentence can have a maximum of 8 words."
    prompt = f"{base_prompt}\nEquipment: {path['object_name']}\nUndesirable Event: {path['event_name']}\n{instruction}"
    fewshot = get_1fewshot_message([base_prompt], [instruction])
    message = fewshot + [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=message,
                    temperature=0.9,
                    top_p=0.9,
                    n=5
                )

    for choice in response.choices:
        output = choice.message.content
        output = process_single_response(output) 
        outputs.append(output)
    return outputs

In [43]:
# Variant prompts
def variant_prompt():
    outputs = []
    fewshot = get_1fewshot_message(base_prompts, instructions)
    for _ in range(5):
        prompt = get_prompt(base_prompts, instructions, path['object_name'], path['event_name'])
        message = fewshot + [{"role": "user", "content": prompt}]
        response = client.chat.completions.create(
                        model="gpt-4o-mini",
                        messages=message,
                        temperature=0.9,
                        top_p=0.9,
                        n=1
                )

        for choice in response.choices:
            output = choice.message.content
            output = process_single_response(output)
            outputs.append(output)
    return outputs

In [57]:
# Demo
same = same_prompt()
variant = variant_prompt()

print ("GPT given same prompt:")
for s in same:
    print(f"- {s}")
print ("GPT given variant prompts:")
for v in variant:
    print(f"- {v}")

GPT given same prompt:
- park brake alarm fault detected
- park brake alarm fault detected
- park brake alarm fault detected
- park brake alarm fault detected
- park brake alarm fault detected
GPT given variant prompts:
- park brake alarm fault detected
- park brake shows alarm fault
- park brake alarm fault detected
- park brake has alarm fault
- park brake alarm fault detected


In [45]:
# Compare diversity over long run
same = []
variant = []
for _ in range(20):
    same.extend(same_prompt())
    variant.extend(variant_prompt())
print (f"Number of unique responses for same prompt: {len(set(same))}")
print (f"Number of unique responses for variant prompts: {len(set(variant))}")

Number of unique responses for same prompt: 3
Number of unique responses for variant prompts: 20


## N completions VS N sentences in ONE completion

In [46]:
# N sentences in ONE completion
def n_sentences():
    outputs = []
    base_prompt = "Generate 5 different Maintenance Work Order (MWO) sentences describing the following equipment and undesirable event."
    instruction = "Each sentence can have a maximum of 8 words."
    prompt = f"{base_prompt}\nEquipment: {path['object_name']}\nUndesirable Event: {path['event_name']}\n{instruction}"
    fewshot = get_5fewshot_message([base_prompt], [instruction])
    message = fewshot + [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=message,
                    temperature=0.9,
                    top_p=0.9,
                    n=1
                )

    for choice in response.choices:
        output = choice.message.content
        output = process_mwo_response(output)
        for sentence in output:
            outputs.append(sentence)
    return outputs

In [47]:
# Demo
n_completion = same_prompt()
n_sentence = n_sentences()

print ("Output in N completions:")
for c in n_completion:
    print(f"- {c}")
print ("Output within 1 completion:")
for s in n_sentence:
    print(f"- {s}")

Output in N completions:
- park brake alarm fault detected
- park brake alarm fault detected
- park brake alarm fault detected
- park brake alarm fault detected
- park brake alarm fault detected
Output within 1 completion:
- park brake alarm fault detected
- park brake shows alarm fault
- alarm fault in park brake system
- park brake has alarm fault
- park brake is experiencing alarm fault


In [48]:
# Compare diversity over long run
n_completion = []
n_sentence = []
for _ in range(20):
    n_completion.extend(same_prompt())
    n_sentence.extend(n_sentences())
print (f"Number of unique responses for N completions: {len(set(n_completion))}")
print (f"Number of unique responses for ONE completion: {len(set(n_sentence))}")

Number of unique responses for N completions: 3
Number of unique responses for ONE completion: 22


## Paraphrase VS Generation

In [60]:
# Paraphrase
def paraphrase():
    outputs = []
    base_prompt = "Generate a Maintenance Work Order (MWO) sentence describing the following equipment and undesirable event."
    instruction = "The sentence can have a maximum of 8 words."
    prompt = f"{base_prompt}\nEquipment: {path['object_name']}\nUndesirable Event: {path['event_name']}\n{instruction}"
    fewshot = get_1fewshot_message([base_prompt], [instruction])
    message = fewshot + [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=message,
                    temperature=0.9,
                    top_p=0.9,
                    n=1
            )

    for choice in response.choices:
        output = choice.message.content
        output = process_single_response(output)
        outputs.append(output)

    keywords = [path['object_name'], path['event_name']]
    paraphrases = paraphrase_mwo(client, output, keywords, 5)
    outputs.extend(paraphrases)
    outputs = list(set(outputs)) # Remove duplicates
    return outputs[:5] # Only return 5 outputs

In [61]:
# Demo
paraphrases = paraphrase()
n_sentence = n_sentences()

print ("Output for Paraphrasing:")
for p in paraphrases:
    print(f"- {p}")
print ("Output for Generation:")
for s in n_sentence:
    print(f"- {s}")

Output for Paraphrasing:
- park brake alarm fault detected
- park brake has an alarm fault identified
- the park brake shows an alarm fault
- detected an alarm fault in the park brake
- an alarm fault in the park brake found
Output for Generation:
- park brake alarm fault detected
- alarm fault in park brake system
- park brake has alarm fault
- park brake system shows alarm fault
- alarm fault present in park brake


In [62]:
# Compare diversity over long run
paraphrases = []
n_sentence = []
for _ in range(20):
    paraphrases.extend(paraphrase())
    n_sentence.extend(n_sentences())
print (f"Number of unique responses for paraphrasing: {len(set(paraphrases))}")
print (f"Number of unique responses for generation: {len(set(n_sentence))}")

Number of unique responses for paraphrasing: 49
Number of unique responses for generation: 20


## Other Paraphraser Models

In [1]:
def model_paraphrase(tokenizer, model, sentences):
    outputs = {}
    for sentence in sentences:
        batch = tokenizer([sentence],truncation=True,padding='longest',max_length=8, return_tensors="pt")
        translated = model.generate(**batch,max_length=15,num_beams=5, num_return_sequences=5, temperature=1.5)
        output = tokenizer.batch_decode(translated, skip_special_tokens=True)
        outputs[sentence] = output
    return outputs

In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")
t5_tokenizer = T5Tokenizer.from_pretrained("t5-large")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-large")
pegasus_tokenizer = PegasusTokenizer.from_pretrained("tuner007/pegasus_paraphrase")
pegasus_model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")

sentences = ["park brake alarm fault detected",
             "alarm fault in park brake system",
             "park brake has alarm fault",
             "park brake system shows alarm fault",
             "alarm fault present in park brake",
             "The boy is walking happily on the street"]

bart_outputs = model_paraphrase(bart_tokenizer, bart_model, sentences)
t5_outputs = model_paraphrase(t5_tokenizer, t5_model, sentences)
pegasus_outputs = model_paraphrase(pegasus_tokenizer, pegasus_model, sentences)

print("BART Outputs:")
for sentence, outputs in bart_outputs.items():
    print(f"{sentence}:")
    for output in outputs:
        print(f"- {output}")
print("T5 Outputs:")
for sentence, outputs in t5_outputs.items():
    print(f"{sentence}:")
    for output in outputs:
        print(f"- {output}")
print("Pegasus Outputs:")
for sentence, outputs in pegasus_outputs.items():
    print(f"{sentence}:")
    for output in outputs:
        print(f"- {output}")

c:\Users\allis\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
c:\Users\allis\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\allis\.cache\huggingface\hub\models--t5-large. Ca

BART Outputs:
park brake alarm fault detected:
- park brake alarm fault detected
- park brake fault detected
- park brake alarm fault fault detected
- park brake alarm fault found
- park brake alarm detected
alarm fault in park brake system:
- alarm fault in park brake
- alarm fault in park brakes
- alarm fault in park car
- alarm fault in park assist
- alarm fault in park system
park brake has alarm fault:
- park brake has alarm fault
- park brake has warning light fault
- park brake has alarm problem
- park brake has alarm fault may cause
- park brake has alarm fault may be
park brake system shows alarm fault:
- park brake system shows alarm fault
- park brake system shows alarm fault in California
- park brake system shows alarm fault in Florida
- park brake system shows alarm fault in China
- park brake system shows alarm fault in car
alarm fault present in park brake:
- alarm may be present in park
- alarm fault present in South Korea
- alarm fault present in South Africa
- alarm 

## Overall Generation Process
For generating sentences for ONE path, we can use the following process:
1. Use multiple **prompt variations**
2. Generate **multiple sentences** for each completion using a random prompt
3. Generate **paraphrases** for each generated sentence